# Regex

## 1. 정규표현식이 필요한 이유 🌐

* **텍스트 정제**: HTML 태그, 공백, `&nbsp;`, 줄바꿈 등 불필요한 요소 제거
* **데이터 추출**: 이메일, 전화번호, URL, 날짜 등 일정한 패턴으로 된 정보 추출
* **유효성 검사**: 크롤링한 데이터가 기대하는 형식(예: 전화번호 `010-xxxx-xxxx`)에 맞는지 검증
* **패턴 기반 필터링**: 원하는 형식의 문자열만 선택적으로 저장 또는 분석

📌 특히 크롤링 후 구조화되지 않은 텍스트에서 **정형화된 데이터 확보**에 필수적인 도구.


## 2. Python `re` 라이브러리 주요 메서드


```python
import re

# search(): 문자열 전체에서 패턴 탐색
m = re.search(r"\d{4}-\d{2}-\d{2}", text)

# match(): 시작 부분이 패턴과 일치하는지
m2 = re.match(r"https?://", url)

# fullmatch(): 전체 문자열이 패턴과 완전 일치하는지
m3 = re.fullmatch(r"[A-Za-z0-9_]+", username)

# findall(): 모든 매칭 결과를 리스트로 반환
dates = re.findall(r"\d{2}/\d{2}/\d{4}", text)

# finditer(): 매칭 결과 iterator 반환
for match in re.finditer(r"\w+@\w+\.\w+", text):
    print(match.group(), match.span())

# sub(): 패턴을 다른 문자열로 치환
clean = re.sub(r"<.*?>", "", html)

# split(): 패턴 기준으로 문자열 분리
parts = re.split(r"\s*,\s*", line)
```

반복 사용 시 **`re.compile()`** 으로 미리 컴파일하면 성능 향상 효과 있음.


## 3. 정규표현식 패턴 만들기 & 자주 쓰는 예제

### 3.1 패턴 구성 요소

* `\d`, `\w`, `\s` 등의 문자 클래스
* `{m,n}`, `+`, `*`, `?` 등의 반복 수량 지정자
* `^`, `$`, `\b` 등 경계 표현
* `.`, `[]`, `()`, `|` 등의 메타문자
* `re.IGNORECASE`, `re.MULTILINE`, `re.DOTALL` 플래그

### 3.2 대표 패턴 예제

| 목적         | 패턴                         | 예시              |
| ---------- | -------------------------- | --------------- |
| 날짜         | `\d{4}-\d{2}-\d{2}`        | `2025-06-19`    |
| 이메일        | `[\w\.-]+@[\w\.-]+\.\w+`   | 이메일 추출          |
| 전화번호       | `01[016789]-\d{3,4}-\d{4}` | `010-1234-5678` |
| URL        | `https?://[^\s<>"]+`       | http/https URL  |
| HTML 태그 제거 | `<.*?>`                    | 비탐욕적 방식         |
| 숫자         | `\d+`                      | 모든 숫자 추출        |

### 3.3 코드 예시

```python
import re

html = "<p>전화: 010-1234-5678</p><a href='https://example.com'>링크</a>"
text = re.sub(r"<.*?>", "", html)
phones = re.findall(r"\b01[016789]-\d{3,4}-\d{4}\b", text)
urls = re.findall(r"https?://[^\s<>']+", html)
```


## 4. 크롤링에서의 정규표현식 활용

```python
from bs4 import BeautifulSoup
import re

html = get_html_from_site()
soup = BeautifulSoup(html, "html.parser")
body = soup.get_text()

emails = re.findall(r"[\w\.-]+@[\w\.-]+\.\w+", body)
links = re.findall(r'href="(https?://[^"]+)"', html)
```

**크롤링 정제 프로세스:**

1. 태그 제거 → 2. 패턴 추출 → 3. 유효성 검사 → 4. 저장


## 5. Excel(openpyxl)에서 정규표현으로 데이터 필터링 🔍

### 5.1 핵심 개념

* `openpyxl`로 Excel 파일을 열고, 셀 단위로 순회하며 데이터를 읽음
* `re`를 이용해 특정 패턴과 매칭되는 값만 추출하거나 정리
* 전처리된 데이터를 **새 시트**나 **파일**로 저장 가능

### 5.2 예제 코드: 이메일 & 전화번호 정제

```python
import re
from openpyxl import load_workbook, Workbook

wb = load_workbook("data.xlsx")
ws = wb.active

email_re = re.compile(r".*?([\w\._%+-]+@[\w\.-]+\.\w+).*?")
phone_re = re.compile(r".*?(01[016789]-\d{3,4}-\d{4}).*?")

wb_out = Workbook()
ws_out = wb_out.active
ws_out.append(["Email", "Phone"])

for row in ws.iter_rows(min_row=2):
    email_val = row[2].value  # Email 컬럼 위치 가정
    phone_val = row[3].value  # Phone 컬럼 위치 가정

    email_match = email_re.match(str(email_val or ""))
    phone_match = phone_re.match(str(phone_val or ""))

    if email_match or phone_match:
        ws_out.append([
            email_match.group(1) if email_match else "",
            phone_match.group(1) if phone_match else ""
        ])

wb_out.save("filtered_openpyxl.xlsx")
```

### 5.3 대용량 Excel 파일의 패턴 추출

```python
from openpyxl import load_workbook
import re, os

pattern = re.compile(r"\d{9}")  # 9자리 숫자
numbers = set()

for fname in os.listdir("xls_files"):
    wb = load_workbook(os.path.join("xls_files", fname), data_only=True)
    for ws in wb.worksheets:
        for row in ws.iter_rows(values_only=True):
            for cell in row:
                if cell:
                    for m in pattern.finditer(str(cell)):
                        numbers.add(m.group())
wb.close()

print(numbers)
```



## 6. 요약 정리

* `re` 모듈: **검색, 추출, 치환, 분리** 기능 제공
* 패턴 설계: 메타문자, 반복자, 경계 표현 조합
* 크롤링 → 정제 → 추출 → **openpyxl 기반 Excel 저장** 흐름
* `openpyxl`을 쓰면 **코드 한 줄로 원하는 데이터만 Excel 파일로 필터링 가능**

---

## ✅ 추천 자료

* `re` 모듈 공식 문서
* openpyxl 공식 튜토리얼 – Excel 작업 자동화
* Medium "Extracting String Patterns…" 




---

### 🔖 참고(인용)
> * [인프런: 파이썬으로 크롤링 시작하기 - 기본편](https://www.inflearn.com/course/python-crawling-basic) 
> * [Python 공식 문서 - re 모듈](https://docs.python.org/3/library/re.html)
> * [Python HOWTO: Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html)
> * [openpyxl 공식 문서](https://openpyxl.readthedocs.io/en/stable/)
> * [Extracting String Patterns from Excel using Python and Regular Expressions - Medium](https://medium.com/@moataz.elmasry/extracting-string-patterns-from-excel-using-python-and-regular-expressions-b290a99f56a3)
